In [2]:
library(sceasy)
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(H5weaver))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(stringr))
library(parallel)

Loading required package: reticulate



In [3]:
meta_data=read.csv('/home//jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-23_fixed.csv')
meta_data<-meta_data %>% filter(Covid_exlcusion=='no',sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 2 Day 0'))


In [4]:
t(table(meta_data$subject.subjectGuid,meta_data$Flu_Year))

      
       BR1001 BR1002 BR1003 BR1004 BR1005 BR1006 BR1007 BR1008 BR1009 BR1010
  2019      1      1      1      1      1      1      1      1      1      1
  2020      0      1      1      1      1      1      1      1      1      1
  2021      0      0      0      0      0      0      0      0      0      0
      
       BR1011 BR1012 BR1013 BR1014 BR1015 BR1016 BR1017 BR1018 BR1019 BR1021
  2019      1      1      1      1      0      0      0      1      0      0
  2020      1      1      1      1      1      1      1      1      1      1
  2021      0      0      0      0      0      1      1      0      1      1
      
       BR1023 BR1024 BR1025 BR1026 BR1028 BR1030 BR1031 BR1032 BR1033 BR1037
  2019      0      0      0      0      0      0      0      0      1      0
  2020      1      1      1      1      1      1      1      1      0      1
  2021      1      1      1      1      1      1      1      1      0      1
      
       BR1040 BR1041 BR1042 BR1043 BR1044 BR1045

# Y1 vs Y2 D0

In [14]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 2 Day 0'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)

    
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+cohort.cohortGuid+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 2 Day 0")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInY2D0',log2FoldChange>0~'HigherInY1D0'))
res$Year<-"Y1"
write.csv(res,paste0("01B_DESeq2_Y1D0_VS_Y2D0_L3/",celltype,"_",Cohort,".csv"))

}



[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "cDC1"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "pDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

In [5]:
meta_data_subset<-meta_data%>% filter(sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 2 Day 0'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)

    
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+cohort.cohortGuid+file.batchID+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 2 Day 0")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInY2D0',log2FoldChange>0~'HigherInY1D0'))
res$Year<-"Y1"
write.csv(res,paste0("01B_DESeq2_Y1D0_VS_Y2D0_L3_with_batch/",celltype,"_",Cohort,".csv"))

}

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

ERROR: Error in eval(expr, p): object 'Cohort' not found


# Y1 vs Y2 D0 by different age group

In [ ]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 2 Day 0'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)

    
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 2 Day 0")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInY2D0',log2FoldChange>0~'HigherInY1D0'))
res$Year<-"Y1"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("01B_DESeq2_Y1D0_VS_Y2D0_L3_by_Cohorts/",celltype,"_",Cohort,".csv"))

}
}


In [8]:
for (Cohort in c("BR1","BR2")){
meta_data_subset<-meta_data%>% filter(cohort.cohortGuid==Cohort,sample.visitName %in% c('Flu Year 1 Day 0','Flu Year 2 Day 0'))
bulk_list<-mclapply(meta_data_subset$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")
bulk <- Seurat:::PseudobulkExpression(object = pbmc, pb.method = 'aggregate',   group.by = "AIFI_L3",slot = 'counts')$RNA
colnames(bulk)<-paste0(x,":",colnames(bulk))
return(bulk)

},mc.cores=60)
    
for (celltype in str_extract(colnames(bulk_list[[1]]), "(?<=:).*")){
print(celltype)

    
matirx_list<-mclapply(bulk_list,function(bulk_matrix){

matching_columns <- grepl(celltype, colnames(bulk_matrix),, fixed = TRUE)
matching_columns<-colnames(bulk_matrix)[matching_columns]
bulk_matrix_single <- as.data.frame(bulk_matrix[, matching_columns, drop = FALSE])
return(bulk_matrix_single)
},mc.cores=60)

df_all<-do.call(cbind,matirx_list)
df_meta_data<-as.data.frame(colnames(df_all))
colnames(df_meta_data)<-'sample_name'
split_columns <- strsplit(df_meta_data$sample_name, ":")
df_meta_data$pbmc_sample_id <- sapply(split_columns, `[`, 1)
df_meta_data$AIFI_L3 <- sapply(split_columns, `[`, 2)
df_meta_data<-left_join(df_meta_data,meta_data_subset)
rownames(df_meta_data)<-df_meta_data$sample_name
selected_gene_list<-read.csv('filtered_gene_list.csv')
selGenes<-selected_gene_list %>% filter(AIFI_L3==celltype) %>% select(gene) %>% pull()  

dds <- DESeqDataSetFromMatrix(df_all[selGenes,], colData = df_meta_data, 
                              design = ~  sample.visitName+
                                          subject.biologicalSex+file.batchID+
                                          CMV.IgG.Serology.Result.Interpretation)
dds <- DESeq(dds,parallel=FALSE)

res <- data.frame(results(dds,contrast=c("sample.visitName", "Flu Year 1 Day 0", "Flu Year 2 Day 0")))
res$celltype<-celltype
res=res %>% arrange(padj) %>% mutate(Direction=case_when(log2FoldChange<0~'HigherInY2D0',log2FoldChange>0~'HigherInY1D0'))
res$Year<-"Y1"
res$cohort.cohortGuid<-Cohort  
write.csv(res,paste0("01B_DESeq2_Y1D0_VS_Y2D0_L3_by_Cohorts_with_batch/",celltype,"_",Cohort,".csv"))

}
}

[1] "ASDC"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Activated memory B cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "Adaptive NK cell"


...

[1] "Adaptive NK cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than

[1] "BaEoMaP cell"


Joining with `by = join_by(pbmc_sample_id)`
converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


ERROR: Error in checkFullRank(modelMatrix): the model matrix is not full rank, so the model cannot be fit as specified.
  One or more variables or interaction terms in the design formula are linear
  combinations of the others and must be removed.

  Please read the vignette section 'Model matrix not full rank':

  vignette('DESeq2')
